# Eikon Data API - How to use StreamingPrices to cache real-time streaming data

The following example demonstrates how to retrieve level 1 streaming data (such as trades and quotes) from Eikon or Refinitiv Workspace. The example will show how to define a StreamingPrices object, which automatically manages a streaming cache available for access at any time. Your application can then reach into this cache and pull out real-time snapshots as Pandas DataFrames by just calling a simple access method.

Using a StreamingPrices object that way prevents your application from sending too many requests to Eikon or Refinitiv Workspace. This is particularly useful if your application needs to retrieve real-time snapshots at regular and short intervals.

## Imports

Imports the 'eikon' library and the 'display' function for this example

In [13]:
import eikon as ek

## Set App Key
Set the AppKey of this application and connect to Eikon or Refinitiv Workspace.

In [15]:
ek.set_app_key('YOUR_APP_KEY')

## Create and open a StreamingPrices object

The StreamingPrices object is created for a list of instruments and fields. The fields parameter is optionnal. If you omit it, StreamingPrices will retrieve all fields available for the requested instruments

In [16]:
streaming_prices = ek.StreamingPrices(
    instruments = ['GBP=','JPY=', 'CAD='], 
    fields   = ['CF_BID','CF_ASK','OPEN_PRC', 'CF_HIGH','CF_LOW', 'CF_CLOSE']
)

The open method tells the StreamingPrices object to subscribe to the streams of the requested instruments.

In [17]:
streaming_prices.open()

<StreamState.Open: 3>

As soon as the open method returns, the StreamingPrices object is ready to be used. Its internal cache is constantly kept updated with the latest streaming information received from Eikon / Refinitiv Workspace. All this happens behind the scene, waiting for your application to pull out data from the cache.  

## Use get_snapshot to pull out data as a DataFrame
Once the StreamingPrices object is opened, you can use the get_snapshot method to pull out data from its internal cache. get_snapshot returns a Pandas DataFrame that uses the same format as the eikon.get_data() function.

get_snapshot can be called any number of times. As these calls return the latest values received from Eikon, several calls to get_snapshot on the same StreamingPrices object may return DataFrames containing different values. Values of the returned DataFrames do not change in real-time, get_snapshot must be called every time your application needs fresh real-time values. 

In [18]:
df = streaming_prices.get_snapshot()
display(df)

,Instrument,CF_BID,CF_ASK,OPEN_PRC,CF_HIGH,CF_LOW,CF_CLOSE
0,GBP=,1.2902,1.2906,1.2922,1.2931,1.2889,1.2922
1,JPY=,108.4500,108.4800,108.5300,108.5700,108.3600,108.5300
2,CAD=,1.3304,1.3306,1.3266,1.3315,1.3262,1.3266


## Get a snapshot for a subset of instruments and fields

In [19]:
df = streaming_prices.get_snapshot(
    instruments = ['JPY=', 'CAD='], 
    fields   = ['CF_BID','CF_ASK']
)
display(df)

,Instrument,CF_BID,CF_ASK
0,JPY=,108.4500,108.4600
1,CAD=,1.3305,1.3306


## Other options to get values from the streaming cache

#### Direct access to real-time fields

In [20]:
print('CAD/BID:', streaming_prices['CAD=']['CF_BID'])
print('JPY/BID:', streaming_prices['JPY=']['CF_BID'])

CAD/BID: 1.3304
JPY/BID: 108.44


#### Direct acces to a Streaming Price instrument

In [21]:
cad = streaming_prices['CAD=']
print(cad.status)
print(cad['CF_BID'])

{'status': <StreamState.Open: 3>, 'code': 'Open', 'message': 'All is well'}
1.3303


#### Iteration on fields

In [22]:
print('CAD=')
for field_name, field_value in streaming_prices['CAD=']:
    print('\t' + field_name + ': ', field_value)
    
print('JPY=')
for field_name, field_value in streaming_prices['JPY=']:
    print('\t' + field_name + ': ', field_value)

CAD=
	CF_BID:  1.3303
	CF_ASK:  1.3307
	OPEN_PRC:  1.3266
	CF_HIGH:  1.3315
	CF_LOW:  1.3262
	CF_CLOSE:  1.3266
JPY=
	CF_BID:  108.44
	CF_ASK:  108.47
	OPEN_PRC:  108.53
	CF_HIGH:  108.57
	CF_LOW:  108.36
	CF_CLOSE:  108.53


#### Iteration on StreamingPrice instruments and fields

In [23]:
for streaming_price in streaming_prices:
    print(streaming_price.name)
    for field_name, field_value in streaming_price:
        print('\t' + field_name + ': ', field_value)

GBP=
	CF_BID:  1.2901
	CF_ASK:  1.2907
	OPEN_PRC:  1.2922
	CF_HIGH:  1.2931
	CF_LOW:  1.2889
	CF_CLOSE:  1.2922
JPY=
	CF_BID:  108.44
	CF_ASK:  108.47
	OPEN_PRC:  108.53
	CF_HIGH:  108.57
	CF_LOW:  108.36
	CF_CLOSE:  108.53
CAD=
	CF_BID:  1.3304
	CF_ASK:  1.3308
	OPEN_PRC:  1.3266
	CF_HIGH:  1.3315
	CF_LOW:  1.3262
	CF_CLOSE:  1.3266


### Close StreamingPrices

In [24]:
streaming_prices.close()

<StreamState.Closed: 1>

Once closed is called the StreamingPrices object stops updating its internal cache. The get_snapshot function can still be called but after the close it always return the same values. 